In [1]:
from transformers import AutoTokenizer
import transformers 
import torch
model = "PY007/TinyLlama-1.1B-step-50K-105b"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    'The TinyLlama project aims to pretrain a 1.1B Llama model on 3 trillion tokens. With some proper optimization, we can achieve this within a span of "just" 90 days using 16 A100-40G GPUs 🚀🚀. The training has started on 2023-09-01.',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

d:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jrtit\.cache\huggingface\hub\models--PY007--TinyLlama-1.1B-step-50K-105b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
d:\anaconda\Lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Result: The TinyLlama project aims to pretrain a 1.1B Llama model on 3 trillion tokens. With some proper optimization, we can achieve this within a span of "just" 90 days using 16 A100-40G GPUs 🚀🚀. The training has started on 2023-09-01.
The goal is: create a small network that learns the language part and predicts its outputs in natural languages with an accuracy >5\% at best for every task (easily trainable without any additional hardware cost). We use BARTSAttentions from Google Cloud Speech Recognition as our base model architecture because it already achieves competitive results by fine tuning them during development phases before release time! This means you are free to experiment or try things out yourself. For more information about what'll be done next please visit https://www.linkedin.com/posts/-miguelbartesattionesgke_projectdavidepra?trk=top-tier-projects&amp;sourceid =8c7fcfaea6#Tip%3D0xA<KEY> <reponame>zhaojieyuanjia/wuxianweixun.github
---
layout : single 
title: "<NAME>

In [ ]:
torch.manual_seed(0)

In [3]:
sequences = pipeline(
    'I wanna kill myself the pizza you delivered is awful and  you should probably eat it',
    do_sample=True,
    top_k=10,
    num_return_sequences=5,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500,
)
for seq in sequences:
    print(f"\n\n\n\n##################\n\n\n\n\nResult: {seq['generated_text']}")





##################




Result: I wanna kill myself the pizza you delivered is awful and  you should probably eat it yourself.
Hmmm...that sounds like fun! What kind of toppings did they have? And how much was it for a single slice? Or would that be cheaper than buying them individually??
I'd never buy individual slices, but if one were available in an affordable size (and my local grocery store doesn't carry this), then maybe $4-$5/piece could go toward meals at restaurants or something else. That seems reasonable enough given what we already paid there anyway. The only thing missing from their listings on Yelp might be some sorta "deliciousness" testimonials as well--but even those aren't very likely either since many places just say things about taste without actually testing out anything with regards to quality etc.. It also looks weird when your favorite restaurant uses different names throughout its reviews: Isolina in New York City; Bella Italia NYC; Fat Daddy Pizzeria; Momof

In [5]:
from transformers import Conversation
pipeline = transformers.pipeline(
    "conversational",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)



In [6]:
conversation = Conversation('I wanna kill myself the pizza you delivered is awful and  you should probably eat it')

In [7]:
replies = pipeline(conversation)

In [8]:
replies

Conversation id: 138abe2d-ae4b-4762-a141-89ca9c1258c7
user: I wanna kill myself the pizza you delivered is awful and  you should probably eat it
assistant: 
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure if you're aware of this but the pizza delivery guy is a real person.
I'm not sure